# Notebook 03: ML Model Training
**Project:** Synthetic Sleep Environment Dataset Generator  
**Authors:** Rushav Dash & Lisa Li  
**Course:** TECHIN 513 — Signal Processing & Machine Learning  
**University:** University of Washington  
**Date:** 2026-02-19

## Table of Contents
1. [Setup & Data Loading](#section-1)
2. [EDA of Sleep Efficiency Dataset](#section-2)
3. [Feature Engineering](#section-3)
4. [Model Training & Cross-validation](#section-4)
5. [Feature Importance Plots](#section-5)
6. [Residual Analysis](#section-6)
7. [Save Models](#section-7)

---
## 1. Setup & Data Loading <a id='section-1'></a>
We import the `SleepQualityModel` class, load the Sleep Efficiency dataset, and configure plotting defaults.

In [ ]:
import sys
import os
import warnings
warnings.filterwarnings('ignore')

PROJECT_ROOT = os.path.abspath(os.path.join(os.getcwd(), '..'))
if PROJECT_ROOT not in sys.path:
    sys.path.insert(0, PROJECT_ROOT)

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import joblib
from sklearn.model_selection import cross_val_score, KFold
from sklearn.metrics import mean_absolute_error, r2_score

from src.data_loader import DataLoader
from src.sleep_quality_model import SleepQualityModel

%matplotlib inline
plt.rcParams.update({'figure.dpi': 120, 'font.size': 11})
sns.set_theme(style='whitegrid')

MODELS_DIR = os.path.join(PROJECT_ROOT, 'models')
os.makedirs(MODELS_DIR, exist_ok=True)

print('Setup complete.')

In [ ]:
loader   = DataLoader(data_dir=os.path.join(PROJECT_ROOT, 'data'))
loader.download_all()
df_sleep = loader.load_sleep_efficiency()

print(f'Sleep Efficiency dataset shape: {df_sleep.shape}')
df_sleep.head()

---
## 2. EDA of Sleep Efficiency Dataset <a id='section-2'></a>
Before training, we inspect the target variable distributions and the full correlation matrix to confirm that the predictors are meaningful.

In [ ]:
print('=== Dataset Overview ===')
print(df_sleep.describe().T.to_string())

### 2.1 Target Variable Distributions
Identify columns that serve as regression targets (sleep efficiency, REM sleep %, sleep duration, awakenings).

In [ ]:
target_candidates = [
    'Sleep efficiency', 'REM sleep percentage',
    'Sleep duration', 'Awakenings'
]
targets = [c for c in target_candidates if c in df_sleep.columns]
print(f'Target columns present: {targets}')

fig, axes = plt.subplots(1, len(targets), figsize=(5 * len(targets), 4))
if len(targets) == 1:
    axes = [axes]

for ax, col in zip(axes, targets):
    sns.histplot(df_sleep[col].dropna(), kde=True, ax=ax, color='steelblue')
    ax.set_title(col, fontsize=10)
    ax.set_xlabel('')

fig.suptitle('Sleep Efficiency Dataset — Target Variable Distributions', fontsize=13)
plt.tight_layout()
plt.show()

### 2.2 Full Correlation Matrix

In [ ]:
numeric_df = df_sleep.select_dtypes(include='number')
corr       = numeric_df.corr()

fig, ax = plt.subplots(figsize=(13, 10))
mask = np.triu(np.ones_like(corr, dtype=bool))
sns.heatmap(
    corr, mask=mask, annot=True, fmt='.2f',
    cmap='coolwarm', center=0, linewidths=0.5,
    annot_kws={'size': 8}, ax=ax
)
ax.set_title('Sleep Efficiency — Full Correlation Matrix', fontsize=13)
plt.tight_layout()
plt.show()

---
## 3. Feature Engineering <a id='section-3'></a>
`SleepQualityModel` exposes two internal preprocessing steps:
- `_encode_categoricals()` — label-encodes string columns (Gender, Smoking status)
- `_engineer_proxy_features()` — creates derived features such as `sleep_debt`, `efficiency_x_duration`, and interaction terms

We inspect the transformed dataframe before training.

In [ ]:
model = SleepQualityModel()

df_encoded = model._encode_categoricals(df_sleep.copy())
print('Columns after categorical encoding:')
print(df_encoded.dtypes)

In [ ]:
df_engineered = model._engineer_proxy_features(df_encoded.copy())
new_cols = [c for c in df_engineered.columns if c not in df_encoded.columns]
print(f'New engineered features ({len(new_cols)}): {new_cols}')
df_engineered[new_cols].describe().T

In [ ]:
if new_cols:
    fig, axes = plt.subplots(1, min(3, len(new_cols)),
                             figsize=(5 * min(3, len(new_cols)), 4))
    if not hasattr(axes, '__len__'):
        axes = [axes]
    for ax, col in zip(axes, new_cols[:3]):
        sns.histplot(df_engineered[col].dropna(), kde=True, ax=ax, color='mediumpurple')
        ax.set_title(col, fontsize=9)
    plt.suptitle('Engineered Feature Distributions', fontsize=12)
    plt.tight_layout()
    plt.show()

---
## 4. Model Training & Cross-validation <a id='section-4'></a>
`SleepQualityModel.train()` fits a separate gradient-boosted regressor for each target variable.  
We use 5-fold CV to estimate generalisation performance (negative MAE and R²).

In [ ]:
print('Training SleepQualityModel...')
model.train(df_sleep)
print('Training complete.')
print(f'Trained targets : {list(model.models_.keys()) if hasattr(model, "models_") else "see model.estimators_"}')

In [ ]:
# Cross-validation on the full pre-processed design matrix
df_cv    = model._engineer_proxy_features(model._encode_categoricals(df_sleep.copy()))
cv_kfold = KFold(n_splits=5, shuffle=True, random_state=42)

cv_results = {}
for target, estimator in model.models_.items():
    feature_cols = [c for c in df_cv.columns if c != target]
    X_cv = df_cv[feature_cols].select_dtypes(include='number').fillna(0)
    y_cv = df_cv[target].fillna(df_cv[target].median())

    mae_scores = -cross_val_score(estimator, X_cv, y_cv,
                                  cv=cv_kfold, scoring='neg_mean_absolute_error')
    r2_scores  =  cross_val_score(estimator, X_cv, y_cv,
                                  cv=cv_kfold, scoring='r2')
    cv_results[target] = {'MAE mean': mae_scores.mean(), 'MAE std': mae_scores.std(),
                          'R2 mean' : r2_scores.mean(),  'R2 std' : r2_scores.std()}
    print(f'{target:40s}  MAE={mae_scores.mean():.4f} ± {mae_scores.std():.4f}'  
          f'  R²={r2_scores.mean():.4f} ± {r2_scores.std():.4f}')

cv_df = pd.DataFrame(cv_results).T
cv_df

In [ ]:
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(12, 4))

cv_df['MAE mean'].plot(kind='barh', ax=ax1, color='steelblue',
                       xerr=cv_df['MAE std'], capsize=4)
ax1.set_title('5-fold CV — Mean Absolute Error')
ax1.set_xlabel('MAE')

cv_df['R2 mean'].plot(kind='barh', ax=ax2, color='darkorange',
                      xerr=cv_df['R2 std'], capsize=4)
ax2.set_title('5-fold CV — R² Score')
ax2.set_xlabel('R²')

plt.suptitle('Cross-validation Performance by Target', fontsize=13)
plt.tight_layout()
plt.show()

---
## 5. Feature Importance Plots <a id='section-5'></a>
Each gradient-boosted regressor exposes `feature_importances_`. We plot the top-15 features for every target to understand which environmental and behavioural variables drive each outcome.

In [ ]:
df_fi = model._engineer_proxy_features(model._encode_categoricals(df_sleep.copy()))

for target, estimator in model.models_.items():
    feature_cols = [c for c in df_fi.columns if c != target]
    X_fi = df_fi[feature_cols].select_dtypes(include='number')

    if not hasattr(estimator, 'feature_importances_'):
        print(f'Skipping {target} — estimator has no feature_importances_')
        continue

    importances = estimator.feature_importances_
    fi_series   = pd.Series(importances, index=X_fi.columns).sort_values(ascending=False)
    top_n       = fi_series.head(15)

    fig, ax = plt.subplots(figsize=(9, 5))
    top_n[::-1].plot(kind='barh', ax=ax, color='steelblue')
    ax.set_title(f'Feature Importance — {target}', fontsize=12)
    ax.set_xlabel('Importance score')
    ax.set_ylabel('')
    plt.tight_layout()
    plt.show()

---
## 6. Residual Analysis <a id='section-6'></a>
Scatter plots of **predicted vs. actual** values (in-sample) and **residuals vs. predicted** help detect systematic bias or heteroscedasticity.

In [ ]:
df_res = model._engineer_proxy_features(model._encode_categoricals(df_sleep.copy()))

for target, estimator in model.models_.items():
    feature_cols = [c for c in df_res.columns if c != target]
    X_res = df_res[feature_cols].select_dtypes(include='number').fillna(0)
    y_res = df_res[target].fillna(df_res[target].median())

    y_pred    = estimator.predict(X_res)
    residuals = y_res.values - y_pred

    in_mae = mean_absolute_error(y_res, y_pred)
    in_r2  = r2_score(y_res, y_pred)

    fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(12, 4))

    ax1.scatter(y_res, y_pred, alpha=0.4, s=15, color='steelblue', edgecolors='none')
    lims = [min(y_res.min(), y_pred.min()), max(y_res.max(), y_pred.max())]
    ax1.plot(lims, lims, 'r--', linewidth=1, label='Perfect fit')
    ax1.set_xlabel('Actual')
    ax1.set_ylabel('Predicted')
    ax1.set_title(f'{target}\nActual vs. Predicted  (MAE={in_mae:.3f}, R²={in_r2:.3f})')
    ax1.legend(fontsize=9)

    ax2.scatter(y_pred, residuals, alpha=0.4, s=15, color='darkorange', edgecolors='none')
    ax2.axhline(0, color='red', linewidth=1, linestyle='--')
    ax2.set_xlabel('Predicted')
    ax2.set_ylabel('Residual (actual - predicted)')
    ax2.set_title(f'{target}\nResiduals vs. Predicted')

    plt.tight_layout()
    plt.show()

### 6.1 Residual Histograms
Ideally the residuals should be approximately Gaussian — a significant skew signals systematic under- or over-prediction.

In [ ]:
n_targets = len(model.models_)
fig, axes = plt.subplots(1, n_targets, figsize=(5 * n_targets, 4))
if n_targets == 1:
    axes = [axes]

for ax, (target, estimator) in zip(axes, model.models_.items()):
    feature_cols = [c for c in df_res.columns if c != target]
    X_r = df_res[feature_cols].select_dtypes(include='number').fillna(0)
    y_r = df_res[target].fillna(df_res[target].median())
    res = y_r.values - estimator.predict(X_r)

    sns.histplot(res, kde=True, ax=ax, color='mediumpurple')
    ax.axvline(0, color='red', linestyle='--')
    ax.set_title(target, fontsize=9)
    ax.set_xlabel('Residual')

fig.suptitle('Residual Distributions by Target', fontsize=13)
plt.tight_layout()
plt.show()

---
## 7. Save Models <a id='section-7'></a>
Persist each trained estimator to the `models/` directory using **joblib** so they can be loaded by the generation pipeline without re-training.

In [ ]:
for target, estimator in model.models_.items():
    safe_name  = target.lower().replace(' ', '_').replace('%', 'pct')
    save_path  = os.path.join(MODELS_DIR, f'{safe_name}_regressor.joblib')
    joblib.dump(estimator, save_path)
    print(f'Saved: {save_path}')

In [ ]:
# Optionally save the full SleepQualityModel object
full_model_path = os.path.join(MODELS_DIR, 'sleep_quality_model.joblib')
joblib.dump(model, full_model_path)
print(f'Full model saved: {full_model_path}')

In [ ]:
# Verify that saved models can be reloaded correctly
loaded_model = joblib.load(full_model_path)
print(f'Loaded model type  : {type(loaded_model)}')
print(f'Loaded model targets: {list(loaded_model.models_.keys())}')
print('\nNotebook 03 complete.')